In [1]:
import tensorflow as tf
from tensorflow.lite.python import interpreter as interpreter_wrapper

2023-02-24 15:26:09.304274: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-24 15:26:10.123818: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model_path="t.tflite"
model_input_shape=(608,608)
interpreter = interpreter_wrapper.Interpreter(model_path=model_path)
interpreter.allocate_tensors()
#model_format = 'TFLITE'

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
#Ehsan input shape correctness
input_details = interpreter.get_input_details()
input_shape = input_details[0]['shape']
input_shape[1] = model_input_shape[0]
input_shape[2] = model_input_shape[1]
interpreter.resize_tensor_input(0, input_shape)
interpreter.allocate_tensors()

In [4]:
# Convert the TensorFlow Lite model to a Keras .h5 model
input_tensors = interpreter.get_input_details()
output_tensors = interpreter.get_output_details()

In [5]:
# Define the input and output shapes of the Keras model
input_shape = (608, 608, 3)
output_shape = [(1, 1, 1, 255), (1, 19, 19, 255), (1, 1, 1, 255)]

In [6]:
@tf.function(input_signature=[tf.TensorSpec(shape=input_shape[i], dtype=tf.float32) for i in range(len(input_shape))])
def tflite_inference(*inputs):
    interpreter.set_tensor(interpreter.get_input_details()[0]['index'], inputs[0])
    interpreter.invoke()
    return tuple(interpreter.get_tensor(output_tensors[i]['index']) for i in range(len(output_tensors)))

In [15]:
print(output_tensors)
[output_tensors[i]['index'] for i in range(len(output_tensors))]
                                        

[{'name': 'StatefulPartitionedCall:0', 'index': 343, 'shape': array([  1,   1,   1, 255], dtype=int32), 'shape_signature': array([ -1,  -1,  -1, 255], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'StatefulPartitionedCall:1', 'index': 301, 'shape': array([  1,  19,  19, 255], dtype=int32), 'shape_signature': array([ -1,  -1,  -1, 255], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}, {'name': 'StatefulPartitionedCall:2', 'index': 322, 'shape': array([  1,   1,   1, 255], dtype=int32), 'shape_signature': array([ -1,  -1,  -1, 255], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantizati

[343, 301, 322]

In [ ]:
# Convert the TensorFlow Lite model to a Keras .h5 model
def m():
    output_tensors = interpreter.get_output_details()
    keras_model = tf.keras.models.Sequential([tf.keras.layers.Lambda(tflite_inference, input_shape=input_shape, output_shape=output_shape)])
    keras_model.save("t.h5")

In [ ]:
if __name__ == '__main__':
    m()